In [ ]:
!pip install -U langchain langchain-google-genai

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

from langchain_core.output_parsers import StrOutputParser

import os
from getpass import getpass

if not os.environ.get("GOOGLE_API_KEY"):
    os.environ["GOOGLE_API_KEY"] = getpass("Enter your Google Gemini API key: ")

llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.1)

response_test = llm.invoke("Who are you?")
print(response_test)

Enter your Google Gemini API key: ··········
content='I am a large language model, trained by Google.' additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []} id='run--2a58bba4-8eb2-4eb4-b21d-8e93534cd36e-0' usage_metadata={'input_tokens': 5, 'output_tokens': 601, 'total_tokens': 606, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 590}}


In [ ]:
# Contract + single-chain smoke test
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate

parser = StrOutputParser()

OUTPUT_CONTRACT_TEXT = """{
  "sentiment": "Positive|Negative|Mixed|Neutral",
  "reason": "string",
  "pros": ["string"],
  "cons": ["string"],
  "rating": { "stars": 1, "out_of": 5, "why": "string" },
  "summary_bullets": ["string", "string", "string", "string"],
  "auto_reply": "string"
}"""

sentiment_pt = PromptTemplate.from_template(
    """You are a precise information extractor.
- Obey this OUTPUT_CONTRACT strictly:
{schema}
- Be terse. No extra words or headings unless asked.

Task: Return the overall sentiment for the REVIEW as one of: Positive, Negative, Mixed, or Neutral.
Output: ONE WORD ONLY (no punctuation).
REVIEW: {review}
"""
).partial(schema=OUTPUT_CONTRACT_TEXT)  # inject schema safely


sentiment_chain = sentiment_pt | llm | parser

# Smoke test on sample
sample_review = {"review":"I love the battery life and display quality, but the camera is needs little work. However apps running like butter"}
print("SENTIMENT:", sentiment_chain.invoke(sample_review))


SENTIMENT: Positive


In [ ]:
# --- Reason (1 brief line) ---
reason_pt = PromptTemplate.from_template(
    """You are a precise information extractor.
- Obey this OUTPUT_CONTRACT strictly:
{schema}
- Be terse. No extra words or headings unless asked.

Task: Give a one-line reason for the sentiment (what tipped the balance).
Output: One short sentence, no lists.
REVIEW: {review}
"""
).partial(schema=OUTPUT_CONTRACT_TEXT)
reason_chain = reason_pt | llm | parser

# --- Pros (JSON array) ---
pros_pt = PromptTemplate.from_template(
    """You are a precise information extractor.
- Obey this OUTPUT_CONTRACT strictly:
{schema}
- Be terse. No extra words or headings unless asked.

Task: Extract clear Pros from the REVIEW.
Output: JSON array of short phrases only.
REVIEW: {review}
"""
).partial(schema=OUTPUT_CONTRACT_TEXT)
pros_chain = pros_pt | llm | parser

# --- Cons (JSON array) ---
cons_pt = PromptTemplate.from_template(
    """You are a precise information extractor.
- Obey this OUTPUT_CONTRACT strictly:
{schema}
- Be terse. No extra words or headings unless asked.

Task: Extract clear Cons from the REVIEW.
Output: JSON array of short phrases only.
REVIEW: {review}
"""
).partial(schema=OUTPUT_CONTRACT_TEXT)
cons_chain = cons_pt | llm | parser





# --- Rating (JSON object) ---  ✅ uses schema var to avoid brace parsing
RATING_EXAMPLE = '{"stars": <int 1-5>, "out_of": 5, "why": "<short reason>"}'

rating_pt = PromptTemplate.from_template(
    """You are a precise information extractor.
- Obey this OUTPUT_CONTRACT strictly:
{schema}
- Be terse. No extra words or headings unless asked.

Task: Infer a star rating (1–5) and a brief justification.
Output: JSON object exactly like:
{rating_example}
Rules: Output JSON only. No code fences/backticks.
REVIEW: {review}
"""
).partial(schema=OUTPUT_CONTRACT_TEXT, rating_example=RATING_EXAMPLE)

rating_chain = rating_pt | llm | parser

# --- Summary bullets (3–4 items as JSON array) ---
summary_pt = PromptTemplate.from_template(
    """You are a precise information extractor.
- Obey this OUTPUT_CONTRACT strictly:
{schema}
- Be terse. No extra words or headings unless asked.

Task: Produce 3–4 bullet summary points.
Output: JSON array of 3 or 4 short strings (bullets).
REVIEW: {review}
"""
).partial(schema=OUTPUT_CONTRACT_TEXT)
summary_chain = summary_pt | llm | parser

In [ ]:
sample = {"review":"I love the battery life and display quality, but the camera is disappointing and the app keeps crashing."}
print("SENTIMENT:", sentiment_chain.invoke(sample))
print("REASON:", reason_chain.invoke(sample))
print("PROS:", pros_chain.invoke(sample))
print("CONS:", cons_chain.invoke(sample))
print("RATING:", rating_chain.invoke(sample))
print("SUMMARY:", summary_chain.invoke(sample))


SENTIMENT: Mixed
REASON: The review presents a mixed bag of strong positives and significant negatives, leading to an overall balanced sentiment.
PROS: ```json
[
  "Excellent battery life",
  "High display quality"
]
```
CONS: ```json
[
  "Camera is disappointing",
  "App keeps crashing"
]
```
RATING: {"stars": 3, "out_of": 5, "why": "Good battery and display, but camera is disappointing and app crashes."}
SUMMARY: ```json
[
  "Excellent battery life and display quality.",
  "Camera performance is disappointing.",
  "Frequent app crashes are a significant issue."
]
```


In [ ]:
import re, json
from langchain_core.runnables import RunnableParallel, RunnableLambda, RunnablePassthrough

def coerce_json(s: str):
    """Accepts strings with code fences or extra prose; returns parsed JSON obj/array."""
    if not isinstance(s, str):
        raise TypeError("coerce_json expected a string")
    txt = s.strip()
    # strip ```json ... ``` or ``` ... ```
    txt = re.sub(r"^```(?:json)?\s*|\s*```$", "", txt, flags=re.I | re.M)
    # grab the first JSON object or array
    m = re.search(r"(\{.*?\}|\[.*?\])", txt, flags=re.S)
    if not m:
        raise ValueError(f"No JSON object/array found in: {s[:80]}...")
    return json.loads(m.group(1))

# fan-out (same as before)
fanout = RunnableParallel({
    "sentiment": sentiment_chain,
    "reason":    reason_chain,
    "pros_raw":  pros_chain,
    "cons_raw":  cons_chain,
    "rating_raw":   rating_chain,
    "summary_raw":  summary_chain,
    "review": RunnablePassthrough()
})



def assemble(d):
    print("d", d)
    return {
        "sentiment": d["sentiment"].strip(),
        "reason": d["reason"].strip(),
        "pros": coerce_json(d["pros_raw"]),
        "cons": coerce_json(d["cons_raw"]),
        "rating": coerce_json(d["rating_raw"]),
        "summary_bullets": coerce_json(d["summary_raw"]),
        "review": d["review"]["review"] if isinstance(d["review"], dict) else d["review"]
    }

assembled = fanout | RunnableLambda(assemble)

# smoke test
sample = {"review":"I love the battery life and display quality, but the camera is disappointing and the app keeps crashing."}
res = assembled.invoke(sample)
import json as _json
print(_json.dumps(res, indent=2))


d {'sentiment': 'Mixed', 'reason': 'The user appreciates the battery and display but is let down by the camera and app stability.', 'pros_raw': '```json\n["Excellent battery life", "High display quality"]\n```', 'cons_raw': '```json\n[\n  "Disappointing camera",\n  "App keeps crashing"\n]\n```', 'rating_raw': '{"stars": 3, "out_of": 5, "why": "Good battery and display, but camera is disappointing and app crashes."}', 'summary_raw': '```json\n[\n  "Excellent battery life and display quality.",\n  "Camera performance is disappointing.",\n  "The app frequently crashes."\n]\n```', 'review': {'review': 'I love the battery life and display quality, but the camera is disappointing and the app keeps crashing.'}}
{
  "sentiment": "Mixed",
  "reason": "The user appreciates the battery and display but is let down by the camera and app stability.",
  "pros": [
    "Excellent battery life",
    "High display quality"
  ],
  "cons": [
    "Disappointing camera",
    "App keeps crashing"
  ],
  "rati

In [ ]:
from langchain_core.runnables import RunnableBranch, RunnableParallel, RunnableLambda

# Four tiny reply chains (short, no fluff)
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
parser = StrOutputParser()

reply_positive = (PromptTemplate.from_template(
    "You are a polite brand rep.\nREVIEW: {review}\nWrite a brief warm thank-you replying to a positive review."
) | llm | parser)

reply_negative = (PromptTemplate.from_template(
    "You are a polite brand rep.\nREVIEW: {review}\nWrite a brief empathetic apology and ask for details to fix issues."
) | llm | parser)

reply_mixed = (PromptTemplate.from_template(
    "You are a polite brand rep.\nREVIEW: {review}\nWrite a brief balanced reply: thank for positives, apologise for issues, ask for 1–2 specifics."
) | llm | parser)

reply_neutral = (PromptTemplate.from_template(
    "You are a polite brand rep.\nREVIEW: {review}\nWrite a brief neutral professional acknowledgment."
) | llm | parser)

# Branch by sentiment (first-match wins); neutral as safe fallback
reply_branch = RunnableBranch(
    (lambda x: "positive" in x["sentiment"].lower(), reply_positive),
    (lambda x: "negative" in x["sentiment"].lower(), reply_negative),
    (lambda x: "mixed"    in x["sentiment"].lower(), reply_mixed),
    (lambda x: "neutral"  in x["sentiment"].lower(), reply_neutral),
    reply_neutral
)

# Compose with your assembled extractor output to add auto_reply
final_pipeline = (assembled | RunnableParallel({
    "sentiment":       RunnableLambda(lambda x: x["sentiment"]),
    "reason":          RunnableLambda(lambda x: x["reason"]),
    "pros":            RunnableLambda(lambda x: x["pros"]),
    "cons":            RunnableLambda(lambda x: x["cons"]),
    "rating":          RunnableLambda(lambda x: x["rating"]),
    "summary_bullets": RunnableLambda(lambda x: x["summary_bullets"]),
    "auto_reply": (RunnableParallel({
        "review":    RunnableLambda(lambda x: x["review"]),
        "sentiment": RunnableLambda(lambda x: x["sentiment"]),
    }) | reply_branch),
}))

# Smoke test
sample = {"review":"I love the battery life and display quality, but the camera is disappointing and the app keeps crashing."}
final_res = final_pipeline.invoke(sample)

import json as _json
print(_json.dumps(final_res, indent=2))


{
  "sentiment": "Mixed",
  "reason": "The user appreciates the battery and display but is let down by the camera and app stability.",
  "pros": [
    "Excellent battery life",
    "High display quality"
  ],
  "cons": [
    "Camera is disappointing",
    "App keeps crashing"
  ],
  "rating": {
    "stars": 3,
    "out_of": 5,
    "why": "Good battery and display, but camera disappoints and app crashes."
  },
  "summary_bullets": [
    "Excellent battery life and display quality.",
    "Camera performance is disappointing.",
    "Frequent app crashes are a significant issue."
  ],
  "auto_reply": "Thank you for taking the time to share your feedback! We're delighted to hear you're loving the battery life and display quality.\n\nWe're truly sorry to learn about your disappointment with the camera and the issues you're experiencing with the app crashing. To help us investigate, could you please tell us a bit more about what you find disappointing with the camera, and perhaps the specific